In [ ]:
!pip install "setuptools<70.0" -q
!pip install brevitas -q

import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 38.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import torch, torchvision, numpy
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("Numpy:", numpy.__version__)

Torch: 2.6.0+cu124
Torchvision: 0.21.0+cu124
Numpy: 1.26.4


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from brevitas.nn import QuantLinear, QuantReLU
from brevitas.quant import Int8WeightPerTensorFixedPoint as WeightQuant
from brevitas.quant import Int8ActPerTensorFixedPoint as ActQuant
from torch.utils.data import DataLoader
import numpy as np
import os

BATCH_SIZE = 64
EPOCHS = 5
LEARNING_RATE = 1e-3
SAVE_DIR = 'quantized_params'

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

class QuantMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = QuantLinear(28 * 28, 256, bias=True, weight_quant=WeightQuant)
        self.relu1 = QuantReLU(act_quant=ActQuant)
        self.fc2 = QuantLinear(256, 128, bias=True, weight_quant=WeightQuant)
        self.relu2 = QuantReLU(act_quant=ActQuant)
        self.fc3 = QuantLinear(128, 10, bias=True, weight_quant=WeightQuant)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

model = QuantMLP()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(EPOCHS):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

layer_idx = 1
for name, module in model.named_modules():
    if isinstance(module, QuantLinear):
        w = module.quant_weight().int().cpu().numpy().astype(np.int8)

        b = module.bias.detach().cpu().numpy()
        b_scaled = (b * 256).astype(np.int32)

        np.save(os.path.join(SAVE_DIR, f'layer{layer_idx}_weight.npy'), w)
        np.save(os.path.join(SAVE_DIR, f'layer{layer_idx}_bias.npy'), b_scaled)

        print(f"Layer {layer_idx}: bias min/max = {b_scaled.min()} / {b_scaled.max()}")
        layer_idx += 1


100%|██████████| 9.91M/9.91M [00:02<00:00, 4.56MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 244kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.74MB/s]
/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:1624: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  return super().rename(names)


Epoch [1/5], Loss: 0.1786
Epoch [2/5], Loss: 0.1032
Epoch [3/5], Loss: 0.2099
Epoch [4/5], Loss: 0.0182
Epoch [5/5], Loss: 0.0011
Test Accuracy: 97.64%
Layer 1: bias min/max = -33 / 42
Layer 2: bias min/max = -31 / 43
Layer 3: bias min/max = -25 / 28


In [ ]:
w1 = np.load('quantized_params/layer1_weight.npy')
b1 = np.load('quantized_params/layer1_bias.npy')

print("Shape of layer1 weights:", w1.shape)
print("First few weights:\n", w1[:5])

print("\nShape of layer1 bias:", b1.shape)
print("First few biases:\n", b1[:5])

print("\nDtype of weights:", w1.dtype)
print("Dtype of biases:", b1.dtype)

Shape of layer1 weights: (256, 784)
First few weights:
 [[ 2  3 -1 ... -3  3  1]
 [-2  0  5 ...  3 -1 -3]
 [-2  3  0 ...  0 -3 -4]
 [-2 -1 -3 ...  0 -3 -2]
 [-2  3  2 ... -2 -2 -4]]

Shape of layer1 bias: (256,)
First few biases:
 [ 17 -12  17  -1  26]

Dtype of weights: int8
Dtype of biases: int32


In [ ]:
b1[:]

array([ 17, -12,  17,  -1,  26,  18,   1,   7, -16,   1,  -1,   7, -12,
        16,  10,   6,  12,  33, -11,  -4,  -5,   5,   8,  14,  17, -15,
         5, -20,  -8,  -3, -10,  19,   0,   9,  12, -11,  23,   7,  11,
        -2,  10,   5,   4,   0,  24, -19,   1,   0,   2, -10,  21,  16,
         3,  18,  -8,  -6,  15,   1,  17,   7,  -7,  17,   2,  22,  23,
        28,   5,  -6,  18,   7,  11,  19,  -3, -14,  -2,  27,  16,   3,
        17, -18,  14,   1,  15, -10,  -9,  -2,  10, -12,   3,   0,  11,
       -11,  -6, -13,  14,  11, -12,   9, -33,  17,  -7, -10,   0,  13,
         2,   8,   6,  19,   1, -14,   0,   6,   9,  12,  21,  18,  22,
         3,  -8,   4, -13, -10,   2,  24, -23,  27,  -4,  -8,  -7,  -9,
         0,  12,   1,   4,  26,   0,  -2,   1,   8,   3,   8,  20, -14,
        -1,   1,  18,   3,   5,  -1,  -6,   2,  17,   7,  17,  28,  11,
        -4,   6,   2,  -8,  22,  22,   8,  -8,   2,  29,   1,  10,   6,
         4,   7,   0,  14,  20, -12,  18,  10,  -5,  -9,  -3,  1

In [ ]:
SAVE_INFER_DIR = 'inference_samples'
os.makedirs(SAVE_INFER_DIR, exist_ok=True)

model.eval()

samples_to_save = 10
saved = 0

with torch.no_grad():
    for images, labels in test_loader:
        for i in range(images.size(0)):
            image = images[i:i+1].to(device)
            label = labels[i].item()

            x_int = (image.view(-1) * 255).to(torch.int8).cpu().numpy()

            output = model(image)
            output_int = output.squeeze().to(torch.int32).cpu().numpy()

            pred = int(torch.argmax(output).item())

            np.savez(os.path.join(SAVE_INFER_DIR, f'sample_{saved}.npz'),
                     input=x_int,
                     output=output_int,
                     true_label=label,
                     pred_label=pred)

            saved += 1
            if saved >= samples_to_save:
                break
        if saved >= samples_to_save:
            break

print(f"Сохранено {saved} инференс-примеров в папке {SAVE_INFER_DIR}")

Сохранено 10 инференс-примеров в папке inference_samples


In [ ]:
data = np.load('inference_samples/sample_4.npz')

print("Входные данные:")
print(data['input'][:20])  # int8, shape (784,)

print("\nВыход сети:")
print(data['output'])  # int32, shape (10,)

print("\nИстинный класс:", data['true_label'])
print("Предсказанный класс:", data['pred_label'])

print("\nТипы данных:")
print("input dtype:", data['input'].dtype)
print("output dtype:", data['output'].dtype)

Входные данные:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Выход сети:
[ -2  -9  -3 -10  12 -10  -4   2  -8   4]

Истинный класс: 4
Предсказанный класс: 4

Типы данных:
input dtype: int8
output dtype: int32


In [ ]:
import shutil

archive_name = 'quantized_mlp_outputs'

shutil.make_archive(archive_name, 'zip', root_dir='.', base_dir='quantized_params')
shutil.make_archive(archive_name + '_inference', 'zip', root_dir='.', base_dir='inference_samples')


'/content/quantized_mlp_outputs_inference.zip'

In [ ]:
def get_safe_scale(module):
    if hasattr(module, 'scale') and callable(module.scale):
        scale = module.scale()
        if scale is not None:
            return scale.detach().cpu().numpy()
    return 1.0

def get_layer_scales(model):
    requant_scales = {}
    layers = list(model.named_modules())
    for i, (name, module) in enumerate(layers):
        if isinstance(module, QuantLinear):
            scale_weight = get_safe_scale(module.weight_quant)

            scale_in = 1.0
            if i > 0:
                _, prev_module = layers[i - 1]
                scale_in = get_safe_scale(prev_module)

            scale_out = 1.0
            if i + 1 < len(layers):
                _, next_module = layers[i + 1]
                scale_out = get_safe_scale(next_module)

            requant_scale = (scale_in * scale_weight) / scale_out
            requant_scales[name] = requant_scale
            print(f"{name}: requant_scale = {requant_scale}")

    return requant_scales

requant_scales = get_layer_scales(model)


fc1: requant_scale = 0.0078125
fc2: requant_scale = 0.0078125
fc3: requant_scale = 0.00390625
